Complete pipeline with evals

In [1]:
from transformers import pipeline
import torch
if torch.cuda.is_available():
    device = "cuda"
    torch_dtype = torch.float16
else:
    device = "cpu"
    torch_dtype = torch.float32
pipe = pipeline(
    "automatic-speech-recognition",
    model = "openai/whisper-small",
    torch_dtype = torch_dtype,
    device = device
)


2025-05-22 08:09:36.916828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747901377.106464      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747901377.161702      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cuda


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset
dataset = load_dataset("mozilla-foundation/common_voice_13_0", "dv",split = "test" )


common_voice_13_0.py:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


n_shards.json:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

...


Reading metadata...: 2212it [00:00, 84068.51it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 16395it [00:00, 97813.06it/s][A


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1653it [00:00, 109440.81it/s]


In [5]:
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

all_preds = []
for preds in tqdm(
    pipe(
        KeyDataset(dataset,"audio"),
        batch_size = 4,
        generate_kwargs = {"task":"transcribe"},
        max_new_tokens = 128
    ),
    total = len(dataset)
):
    all_preds.append(preds["text"])

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(
  0%|          | 0/2212 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
100%|██████████| 2212/2212 [15:44<00:00,  2.34it/s]


In [9]:
!pip install -q evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.9 MB/s eta 0:00:0000:0100:01


In [13]:
from evaluate import load
wer_init = load("wer")
wer_ortho = 100 * wer_init.compute(references = dataset["sentence"], predictions= all_preds)
print(f"WER prior to normalization {wer_ortho} %")

WER prior to normalization 150.63723100494462 %


In [14]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
normalizer = BasicTextNormalizer()

normalized_reference= [ normalizer(text) for text in dataset["sentence"]]
normalized_preds = [normalizer(text) for text in all_preds]

filtered_norm_preds = [normalized_preds[i] for i in range(len(normalized_preds)) if len(normalized_preds[i]) >0]
filtered_norm_ref = [normalized_reference[i] for i in range(len(normalized_reference)) if len(normalized_reference[i]) >0]

In [17]:
wer_norm = wer_init.compute(references = filtered_norm_ref, predictions =filtered_norm_preds  )
print(f"WER after the normalization {wer_norm} %")

WER after the normalization 1.1043745870570643 %
